In [1]:
import cv2
import numpy as np
import math
from PIL import Image

In [13]:
def unwrap_fisheye1(input_image_path, output_image_path):
    # Read the fisheye image
    fisheye_image = cv2.imread(input_image_path)

    # Get image dimensions
    height, width = fisheye_image.shape[:2]

    # Define the fisheye camera parameters
    K = np.array([[500, 0, width // 2],
                  [0, 500, height // 2],
                  [0, 0, 1]])
    K = K.astype(np.float32)

    # Create an empty distortion coefficients array
    D = np.zeros((4, 1), dtype=np.float32)
    fisheye_image = cv2.resize(fisheye_image, (200, 200))

    # Undistort the fisheye image
    undistorted_image = cv2.fisheye.undistortImage(fisheye_image, K, D)

    # Save the undistorted image
    cv2.imwrite(output_image_path, undistorted_image)



In [14]:

input_image_path = "Images/unwrapped_walgreens_left.jpeg"
output_image_path = "Images_undistorted/undistorted_image.jpg"
unwrap_fisheye1(input_image_path, output_image_path)


In [11]:
def unwrap_fisheye(input_image_path, output_image_path):
     # Read the fisheye image
    fisheye_image = cv2.imread(input_image_path)

    # Get image dimensions
    height, width = fisheye_image.shape[:2]
    
    # define field of view
    fov = 190
    
    # Create a blank image for the unwrapped equirectangular projection
    unwrapped_image = np.zeros((height, width, 3), dtype=np.uint8)

    
    for x in range(width):
        for y in range(height):
            
            # Projection to 3D point, preparation of angles
            theta_s = math.radians(fov * x / width - 0.5)
            phi_s = math.radians(fov * y / height - 0.5)
            
            # Point in 3D
            x1 = math.cos(phi_s) * math.sin(theta_s)
            y1 = math.cos(phi_s) * math.cos(theta_s)
            z1 = math.sin(phi_s)
            
            # Computation of rho, distance from center of projection to point
            root = math.sqrt(x1**2 + z1 ** 2)
            rho = height / fov * math.atan(root / y1)
            theta = math.atan(z1 / x1)
            
            # Projected point
            x2 = int(0.5 * width + rho * math.cos(theta))
            y2 = int(0.5 * height + rho * math.sin(theta))
            
            # Map the pixel value from fisheye image to the unwrapped image
            if 0 <= x2 < width and 0 <= y2 < height:
                unwrapped_image[y, x] = fisheye_image[int(y2), int(x2)]
    
    # Save the unwrapped image
    cv2.imwrite(output_image_path, unwrapped_image)
    

In [12]:

input_image_path = "Images/test.jpg"
output_image_path = "Images_undistorted/test_undistorted.jpg"
unwrap_fisheye(input_image_path, output_image_path)
